CÉLULA DE MARKDOWN EXEMPLO

## Projeto IIA ##
* Professor Díbio , Turma 01 , 2025.1

# Autores: #
* Marcelo Marques Rodrigues - 221018960

favor rodar{ pip install -r requirements.txt }

## Parte 1: Processar Geolocalizações e Mostrar em Mapa ##

In [1]:
import os
import pandas as pd
from geopy.distance import geodesic

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import folium
import time


Processamos o Dataset

In [2]:
caminho_arquivo_csv = os.path.join( "..", "data", "associacoes_geocoords.csv" )

try:
    df_associacoes = pd.read_csv(caminho_arquivo_csv)

except FileNotFoundError:
    print('Arquivo .csv não encontrado! Verifique o Diretório')
    df_associacoes = pd.DataFrame( columns=['Nome_Associacao', 'Localidade_Principal_Projeto', 'Latitude', 'Longitude'] )

print("Dataframe Carregado")
print(df_associacoes.head(6))
print(f'\n Número Total de Associações: {len(df_associacoes)}')

Dataframe Carregado
                                     Nome_Associacao  \
0  Associação de Agricultores Familiares da Eco C...   
1  Associação Agrícola do Distrito Federal e Ride...   
2  Associação Mista dos Agricultores Familiares, ...   
3  Associação dos Produtores da Agricultura Famil...   
4  Associação dos Produtores Rurais de Alexandre ...   
5  Associação dos Produtores de Hortigranjeiros d...   

  Localidade_Principal_Projeto  Latitude  Longitude  
0                São Sebastião -15.88007  -47.77402  
1                   Taguatinga -15.83322  -48.05677  
2                  Santa Maria -16.00157  -47.98586  
3                        Guará -15.81459  -47.98497  
4                   Brazlândia -15.67693  -48.20507  
5                         Gama -16.01609  -48.07178  

 Número Total de Associações: 17


Calculamos distância e encontramos as associações mais próximas comparadas à entrada do usuário

In [3]:
def calcular_distancia_km(coord_usuario, coord_associacao): # calculamos a distancia por coordenadas do usuario com a associação

    if None in coord_usuario or pd.isna(coord_usuario[0]) or pd.isna(coord_usuario[1]) or \
       None in coord_associacao or pd.isna(coord_associacao[0]) or pd.isna(coord_associacao[1]):
        return float('inf') # retorna infinito se a coordenada do usuária for none ou nan
    
    return geodesic(coord_usuario, coord_associacao).km

def encontrar_associacoes_proximas( df_associacoes, coord_usuario_lat, coord_usuario_lon, max_distancia_km=None, top_n=None):
    
    if df_associacoes.empty:
        print(" DataFrame vazio. Não há associação para encontrar" )
        return pd.DataFrame()

    coord_usuario = (coord_usuario_lat, coord_usuario_lon)

    df_resultado = df_associacoes.copy()

    df_resultado['Latitude'] = pd.to_numeric(df_resultado['Latitude'], errors='coerce') # tratamento
    df_resultado['Longitude'] = pd.to_numeric(df_resultado['Longitude'], errors='coerce') #tratamento
    df_resultado['Distancia_km'] = df_resultado.apply(
        lambda row: calcular_distancia_km(coord_usuario, (row['Latitude'], row['Longitude'])), # lambda permite calcular todas as dis^tanias
        axis=1
    )

    df_resultado = df_resultado.sort_values(by='Distancia_km') # sort

    if max_distancia_km is not None:
        df_resultado = df_resultado[df_resultado['Distancia_km'] <= max_distancia_km]
    if top_n is not None: # melhores N recomendações
        df_resultado = df_resultado.head(top_n)

    return df_resultado

Teste da função

In [4]:
def geocodificar_endereco_brasilia(endereco, tentativas=3, delay=2):

    geolocator = Nominatim(user_agent="projeto_iia") 
    endereco_completo = f"{endereco}, Brasília, DF, Brazil"
    print(f"Tentando geocodificar: '{endereco_completo}'")
    for tentativa in range(tentativas):
        try:
            location = geolocator.geocode(endereco_completo, timeout=10) # responder dentro do timeout
            if location:
                print(f"Endereço encontrado: {location.address}")
                return location.latitude, location.longitude
            else:
                print("Endereço não encontrado.")
                return None
        except GeocoderTimedOut:
            print(f"Tentativa {tentativa + 1} de {tentativas}: Timeout na geocodificação. Tentando novamente em {delay}s...")
            time.sleep(delay)
        except GeocoderUnavailable:
            print(f"Tentativa {tentativa + 1} de {tentativas}: Serviço de geocodificação indisponível. Tentando novamente em {delay}s...")
            time.sleep(delay)
        except Exception as e:
            print(f"Um erro inesperado ocorreu durante a geocodificação: {e}")
            return None
    print("Falha na geocodificação após múltiplas tentativas.")
    return None

lat_usuario , lon_usuario = geocodificar_endereco_brasilia('AV PARQUE AGUAS CLARAS N 3305' , 3 , 2) # eNTRADA dO eNDEREÇO

associacoes_proximas = encontrar_associacoes_proximas(
    df_associacoes,
    lat_usuario,
    lon_usuario,
    max_distancia_km= 20, # raio do filtro
)

if not associacoes_proximas.empty:
    print(f"\n Associações encontradas perto de ({lat_usuario} , {lon_usuario}):")
    colunas_para_mostrar = ['Nome_Associacao', 
                            'Localidade_Principal_Projeto',
                            'Distancia_km']
    print(associacoes_proximas[colunas_para_mostrar])
else:

    print(f"\nNenhuma associação encontrada dentro dos critérios perto do usuário.")

# sem filtro de distância máxima, apenas as 3 mais próximas
print("\n--------------------------------------------------")
associacoes_topn = encontrar_associacoes_proximas (
    df_associacoes,
    lat_usuario,
    lon_usuario,
    top_n = 3
)

if not associacoes_topn.empty:
    print(f"\n Top 3 associações mais próximas de ({lat_usuario}, {lon_usuario}):")
    colunas_para_mostrar = ['Nome_Associacao', 'Localidade_Principal_Projeto', 'Distancia_km',]
    print(associacoes_topn[colunas_para_mostrar])
else:
    print(f"\nNenhuma associação encontrada perto do usuário.")

Tentando geocodificar: 'AV PARQUE AGUAS CLARAS N 3305, Brasília, DF, Brazil'
Endereço encontrado: Avenida Parque Águas Claras, Águas Claras, Região Geográfica Imediata do Distrito Federal, Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Distrito Federal, Região Centro-Oeste, 71910-360, Brasil

 Associações encontradas perto de (-15.8364452 , -48.0217648):
                                      Nome_Associacao  \
1   Associação Agrícola do Distrito Federal e Ride...   
3   Associação dos Produtores da Agricultura Famil...   
8   Associação dos Trabalhadores Rurais da Agricul...   
6   Associação dos Produtores Rurais Orgânicos e C...   
15  Cooperativa Agrícola da Região de Planaltina (...   
7   Associação dos Produtores Rurais Novo Horizont...   
11  Cooperativa dos Produtores Rurais de Planaltin...   
2   Associação Mista dos Agricultores Familiares, ...   
14  Cooperativa dos Agricultores Familiares Ecológ...   


In [ ]:
import folium
import openrouteservice
from openrouteservice import convert 
import os
import pandas as pd 
from geopy.distance import geodesic 

endereco_usuario = 'w3 norte, Brasília, DF'  ###############################################  brasilia , alterar só a primeira parte

api_key_ors = "5b3ce3597851110001cf62481e0e6170f26144b7a705d9d5a95084d0"
client_ors = openrouteservice.Client(key=api_key_ors)

# começa a geocodificar
print(f"Geocodificando o endereço do usuário: {endereco_usuario}")
coords_usuario_lat_lon = geocodificar_endereco_brasilia(endereco_usuario) 
if coords_usuario_lat_lon is None:
    print("Não foi possível geocodificar o endereço do usuário. Saindo.")
    
else:
    lat_usuario, lon_usuario = coords_usuario_lat_lon
    

    # -
    if 'df_associacoes' not in locals() or df_associacoes.empty:
        print("DataFrame 'df_associacoes' não encontrado ou está vazio. Carregue os dados das associações primeiro.")
        
    else:
        print("Encontrando a associação mais próxima...")
        associacao_mais_proxima_df = encontrar_associacoes_proximas(
            df_associacoes,
            lat_usuario,
            lon_usuario,
            top_n=1  
        )

        if associacao_mais_proxima_df.empty:
            print("Nenhuma associação encontrada próxima ao usuário.")
            
        else:
            associacao_info = associacao_mais_proxima_df.iloc[0]
            lat_associacao = associacao_info['Latitude']
            lon_associacao = associacao_info['Longitude']
            nome_associacao = associacao_info['Nome_Associacao']
            distancia_reta_km = associacao_info['Distancia_km']
            print(f" {nome_associacao} a {distancia_reta_km:.2f} km (em linha reta).")


            
            # OpenRouteService
            origem_coords_ors = (lon_usuario, lat_usuario)
            destino_coords_ors = (lon_associacao, lat_associacao)
            
            rota_geojson = None
            distancia_rota_km = float('inf')
            duracao_rota_min = float('inf')

            try:
                print("Calculando rota pela estrada...")
                directions = client_ors.directions(
                    coordinates=[origem_coords_ors, destino_coords_ors],
                    profile='driving-car',  
                    format='geojson'
                )
                rota_geojson = directions #salva
                
                if directions['features'] and directions['features'][0]['properties']['segments']:
                    distancia_rota_metros = directions['features'][0]['properties']['segments'][0]['distance']
                    duracao_rota_segundos = directions['features'][0]['properties']['segments'][0]['duration']
                    distancia_rota_km = distancia_rota_metros / 1000
                    duracao_rota_min = duracao_rota_segundos / 60
                    print(f"Distância da rota: {distancia_rota_km:.2f} km, Duração estimada: {duracao_rota_min:.2f} minutos.")
                else:
                    print("Não foi possível extrair detalhes da rota (distância/duração).")

            except openrouteservice.exceptions.ApiError as e:
                print(f"Erro da API OpenRouteService: {e}")
                print("Não foi possível calcular a rota pela estrada. Mostrando linha reta no mapa.")
            except Exception as e:
                print(f"Um erro inesperado ocorreu ao calcular a rota: {e}")
                print("Não foi possível calcular a rota pela estrada. Mostrando linha reta no mapa.")

            
            
            map_center_lat = (lat_usuario + lat_associacao) / 2
            map_center_lon = (lon_usuario + lon_associacao) / 2
            
            mapa_rota_usuario = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=11)

            # Marca o user
            folium.Marker(
                location=[lat_usuario, lon_usuario],
                tooltip="Sua Localização",
                popup=f"<b>Sua Localização</b><br>{endereco_usuario}<br>({lat_usuario:.5f}, {lon_usuario:.5f})",
                icon=folium.Icon(color='blue', icon='user', prefix='fa')
            ).add_to(mapa_rota_usuario)

            # MArca a associação
            popup_associacao_text = (f"<b>{nome_associacao}</b><br>"
                                     f"Dist. reta: {distancia_reta_km:.2f} km<br>"
                                     f"({lat_associacao:.5f}, {lon_associacao:.5f})")
            if distancia_rota_km != float('inf'):
                 popup_associacao_text += f"<br>Dist. rota: {distancia_rota_km:.2f} km ({duracao_rota_min:.0f} min)"

            folium.Marker(
                location=[lat_associacao, lon_associacao],
                tooltip=nome_associacao,
                popup=popup_associacao_text,
                icon=folium.Icon(color='green', icon='shopping-basket', prefix='fa')
            ).add_to(mapa_rota_usuario)

            if rota_geojson and rota_geojson.get('features'):
                folium.GeoJson(
                    rota_geojson,
                        name=f"Rota para {nome_associacao}",
                            style_function=lambda x: {'color': 'red', 'weight': 5, 'opacity': 0.7}
                ).add_to(mapa_rota_usuario)
            else:
                folium.PolyLine(
                    locations=[(lat_usuario, lon_usuario), (lat_associacao, lon_associacao)],
                        color="red",
                            weight=2.5,
                                opacity=0.8,
                    tooltip="Rota pela estrada não disponível (linha reta)"
                ).add_to(mapa_rota_usuario)
            
            folium.LayerControl().add_to(mapa_rota_usuario)
            
            map_file_name = "mapa_rota_usuario_associacao.html"
            map_file_path = os.path.join("..", "data", map_file_name)
            
            try:
                mapa_rota_usuario.save(map_file_path)
                print(f"Mapa salvo em: {os.path.abspath(map_file_path)}")
            except Exception as e:
                print(f"Erro ao salvar o mapa: {e}")


            display(mapa_rota_usuario)

Geocodificando o endereço do usuário: w3 norte, Brasília, DF
Tentando geocodificar: 'w3 norte, Brasília, DF, Brasília, DF, Brazil'
Endereço encontrado: W3 Norte, HCGN 716, Setor Noroeste, Brasília, Plano Piloto, Região Geográfica Imediata do Distrito Federal, Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Distrito Federal, Região Centro-Oeste, 70770-731, Brasil
Encontrando a associação mais próxima...
 Cooperativa dos Produtores Rurais de Planaltina de Goiás e Região (Prorural) a 6.41 km (em linha reta).
Calculando rota pela estrada...
Distância da rota: 8.03 km, Duração estimada: 11.32 minutos.
Mapa salvo em: c:\Users\marce\ProjetoIIA\data\mapa_rota_usuario_associacao.html
